# All pieces together

This is just a notebook that puts together all the pieces of the code that we have been writing during this tutorial.

In [ ]:
!apt-get install python3-mysqldb
!pip install sqlalchemy sql_magic
!pip install flask==0.12.2
!pip install flask-ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, render_template
from flask_ngrok import run_with_ngrok
from sqlalchemy import create_engine
from flask import request
import urllib

import matplotlib
matplotlib.use('Agg')

import pandas as pd

 
app = Flask(__name__, 
            template_folder = '/content/drive/My Drive/templates',
            static_folder  = '/content/drive/My Drive/static'
            )
run_with_ngrok(app)
 
@app.route('/citibike')
def citibike_stations():
 
    conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
        host = 'db.ipeirotis.org', 
        user = 'student',
        db = 'citibike_fall2017',
        password = 'dwdstudent2015',
        encoding = 'utf8mb4')
 
    engine = create_engine(conn_string)
    con = engine.connect()
    stations = con.execute("SELECT DISTINCT id AS station_id, name, capacity, lat, lon  FROM status_fall2017")
    con.close()
 
    return render_template('citibike.html', stations=stations)
 

@app.route('/station_status')
def station_status():

    station_id = int(request.args.get('station_id'))
    station_name = request.args.get('station_name')
    

    conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
        host = 'db.ipeirotis.org', 
        user = 'student',
        db = 'citibike_fall2017',
        password = 'dwdstudent2015',
        encoding = 'utf8mb4')

    engine = create_engine(conn_string)
    con = engine.connect()
    query = '''SELECT available_bikes, 
                      available_docks, 
                      capacity, 
                      ROUND(100*available_bikes/capacity,1) AS percent_full,
                      communication_time 
               FROM status_fall2017
               WHERE id = %s'''
    status = con.execute(query, (station_id,))

    image_filename = create_plot(station_id,con)

    con.close()

    return render_template('station_status.html', 
                           station_id = station_id, 
                           station_name = station_name,
                           statuses=status, 
                           image=image_filename)

@app.route('/')
def home():
    return render_template('search_stations.html')

@app.route('/search/')
def search():

    name = request.args.get('name')

    conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
        host = 'db.ipeirotis.org', 
        user = 'student',
        db = 'citibike_fall2017',
        password = 'dwdstudent2015',
        encoding = 'utf8mb4')
    engine = create_engine(conn_string)

    con = engine.connect()
    query = '''SELECT DISTINCT id AS station_id, name, capacity, lat, lon
               FROM status_fall2017
               WHERE name LIKE %s
            '''
    stations = con.execute(query, ('%'+name+'%',))
    con.close()

    return render_template('citibike.html', stations=stations)


def create_plot(station,con):
    # Read the data from the database
    sql = '''
        SELECT communication_time, ROUND(100*available_bikes/capacity,1) AS percent_full  
        FROM status_fall2017 WHERE id= %(station_id)s
    '''
    status = pd.read_sql(sql, params={"station_id": station}, con=con)
    status.set_index('communication_time', inplace=True)
    # Create a plot, displaying the "percent_full" variable with "communication_time" as the x-axis
    plot = status.plot(
        figsize=(10, 2), 
        grid=True, 
        xlim=('2017-10-01', '2017-10-31'),
        ylim=(0,100),
        legend=False
    )
    # Store the file under the static folder, and give a name plot-<stationid>.png
    filename = 'static/plot-'+str(station)+ '.png'
    fig = plot.get_figure()
    fig.savefig('/content/drive/My Drive/'+filename, dpi=72, pad_inches=1)
    fig.clear()
    # Return back the name of the image file
    return filename


app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://64c05fb71011.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Oct/2020 19:58:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:58:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Oct/2020 19:58:48] "GET /search?name=LAGuardia HTTP/1.1" 308 -
127.0.0.1 - - [20/Oct/2020 19:58:50] "GET /search/?name=LAGuardia HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:58:53] "GET /station_status?station_id=161 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:59:00] "GET /station_status?station_id=161 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:59:00] "GET /static/plot-161.png HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 20:02:55] "GET /station_status?station_id=161 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 20:02:55] "GET /static/plot-161.png HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 20:02:56] "GET /favicon.ico HTTP/1.1" 404 -


**`search_stations.html`**

```html
<!DOCTYPE html>
<html>

<head>
	<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css"
		integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
	<title>Search Citibike Stations</title>
</head>

<body>
	<div class="container">
		<div class="panel panel-info">
			<div class="panel-heading" align="center">
				<h1>
					<form action="/search">
						Station Name<br>
						<input type="text" name="name"><br>
						<input class="btn btn-primary" type="submit" value="Submit">
          </form>
				</h1>
			</div>
		</div>
	</div>
</body>
```

`citibike.html`

```html
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
</head>
<body>  
  <div class="container">
    <div class="panel panel-info">
      <div class="panel-heading">
          <h1 class="panel-title">Citibike Stations</h1>
      </div>
      <table  class="table table-striped table-bordered table-hover">
      <thead>
          <tr> 
            <th>Station ID</th>
            <th>Address</th>
            <th>Capacity</th>
            <th>Lon / Lat</th>
          </tr>
      </thead>
      <tbody>
          {% for station in stations %}    <!-- Fill table rows for each station --> 
              <tr>
                  <td><a href="/station_status?station_id={{ station.station_id }}">{{ station.station_id }}</a></td>
                  <td><a href="/station_status?station_id={{ station.station_id }}">{{ station.name }}</a></td>
                  <td>{{ station.capacity }}</td>
                  <td>{{ station.lon }}, {{ station.lat }}</td>
              </tr>
          {% endfor %}
        </tbody>
      </table>
    </div>
  </div>
</body>
</html>
```

**`station_status.html`**

```html
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
</head>
<body>
  <div class="container">
    <div class="panel panel-info">
      <div class="panel-heading">
        <h1 class="panel-title">Citibike Station {{station_id}} Status Over Time </h1>
      </div>
      
      <img class="center-block" src="{{image}}" />
      
      <table  class="table table-striped table-bordered table-hover">
        <thead>
            <tr>    <!-- Construct table header -->
              <th> Time </th>
              <th> Bikes available </th>
              <th> Docks available </th>
              <th> Total docks </th>
              <th> Percent full </th>
            </tr>
        </thead>
        <tbody>
            {% for status in statuses %}
                <tr>
                  <td>{{ status.communication_time }}</td>
                  <td>{{ status.available_bikes }}</td>
                  <td>{{ status.available_docks }}</td>
                  <td>{{ status.capacity }}</td>
                  <td>{{ status.percent_full }}%</td>
                </tr>
            {% endfor %}
          </tbody>
        </table>
    </div>
  </div>
</body>
</html>
```